In [1]:
import cantera as ct
from omnisoot import ConstantVolumeReactor, SootGas, SootThermo, constants
import numpy as np
import os
import pandas as pd
import time

from scipy import integrate
# import graphviz
from matplotlib import pyplot as plt
from collections import defaultdict

In [2]:
Av = 6.0221408e+23;

In [3]:
gas = ct.Solution('abf_1bar.yaml')

In [4]:
soot_gas = SootGas(gas);

In [5]:
# Just to avoid the launcher
T5 = 1800
P5 = 1e5
simulation_time = 0.1;

In [6]:
IC = T5, P5, {'CH4':0.3, 'N2':0.7}

In [7]:
soot_gas.TPY = IC ;
max_res_time = 1;

In [28]:
### Initial soot properties 
N_agg_number_0 = 1e20; # 1/m3
n_p_0 = 1;
d_p_0 = 3e-9; # m
H_C_ratio = 0.8; 
### Calculating soot array to use as an input to the model
N_agg_0 = N_agg_number_0 / soot_gas.density / constants.Av; # mol/kg
N_pri_0 = N_agg_0 * n_p_0; # mol/kg
mass_c_p_0 = constants.Pi / 6 * d_p_0 ** 3.0 * constants.rho_soot; # kg of carbon
mole_c_p_0 = mass_c_p_0 / constants.MW_carbon; # mole of carbon
C_tot_0 = N_pri_0 * Av * mole_c_p_0;
H_tot_0 = H_C_ratio * C_tot_0;

In [30]:
constUV = ConstantVolumeReactor(soot_gas)

In [31]:
soot = constUV.soot;
# soot.oxidation_enabled = True;
PAH_growth_model_type = ["ReactiveDimerization", "DimerCoalescence", "EBridgeModified", "IrreversibleDimerization"][0]
soot.PAH_growth_model_type = PAH_growth_model_type;
soot.set_precursor_names(['A2', 'A3', 'A4']) #['A2', 'A2R5', 'A3R5', 'A4R5']
particle_dynamics_model_type = "Monodisperse" # "Monodisperse" or "Sectional"
soot.particle_dynamics_model_type = particle_dynamics_model_type;
pdynamics = soot.particle_dynamics_model;
constUV.solver_type = "BDF";
constUV.initial_soot_type = "custom";
constUV.user_defined_initial_soot = np.array([N_agg_0, N_pri_0, C_tot_0, H_tot_0]);
#soot.particle_dynamics_model.beta_interp_method_type = 'harmonic'
soot.particle_dynamics_model.beta_interp_method = 'harmonic'
soot.soot_enabled = True;

### Flags
soot.inception_enabled = False;
soot.PAH_adsorption_enabled = False;
soot.surface_growth_enabled = True;
soot.oxidation_enabled = False;
soot.coagulation_enabled = False;

In [32]:
soot.PAH_growth_model.K_REACT_prefactor = 1.0;
soot.PAH_growth_model.K_PAH_ADS_prefactor = 1.0;

In [33]:
def append_sectional():
    params = [pdynamics.N_agg_sec_all(), pdynamics.N_pri_sec_all(), 
                  pdynamics.n_p_sec_all(), pdynamics.d_p_sec_all(),
                  pdynamics.d_m_sec_all(), pdynamics.d_g_sec_all()];
    for pi, param in enumerate(params):
        for si in range(n_secs): 
            sectional_data[pi*n_secs+si].append(param[si]);

def append():
    states.append(gas.state ,
              N_agg = soot.N_agg, N_pri =soot.N_pri, restime=constUV.restime, 
              C_tot = soot.C_tot, H_tot = soot.H_tot, A_tot = soot.A_tot,
              d_p = soot.d_p, d_m = soot.d_m, d_v = soot.d_v, d_g = soot.d_g,
              n_p = soot.n_p,
              total_mass = soot.total_mass, volume_fraction = soot.volume_fraction, SSA = soot.SSA,
              inception_mass = soot.inception_mass,
              coagulation_mass = soot.coagulation_mass,
              PAH_adsorption_mass = soot.PAH_adsorption_mass,
              surface_growth_mass = soot.surface_growth_mass,
              oxidation_mass = soot.oxidation_mass,
              total_carbon_mass = constUV.total_carbon_mass,
              total_hydrogen_mass = constUV.total_hydrogen_mass,
              soot_carbon_mass = constUV.soot_carbon_mass,
              removed_gas_mass = constUV.removed_gas_mass,
             );
    
    if particle_dynamics_model_type == "Sectional": 
        append_sectional();

In [34]:
states = ct.SolutionArray(gas, 1, extra={'restime': [0.0] , 
                                     'N_agg': [soot.N_agg], 'N_pri': [soot.N_pri], 
                                     'C_tot': [soot.C_tot], 'H_tot': [soot.H_tot],
                                     'A_tot': [soot.A_tot],
                                     'd_p': [soot.d_p], 'd_m': [soot.d_m],
                                     'd_v': [soot.d_v], 'd_g': [soot.d_g],
                                     'n_p': [soot.n_p],
                                     'total_mass' : [soot.total_mass],
                                     'volume_fraction':[soot.volume_fraction],
                                     'SSA': [soot.SSA],
                                     'inception_mass': [soot.inception_mass],
                                     'coagulation_mass': [soot.coagulation_mass],
                                     'PAH_adsorption_mass' : [soot.PAH_adsorption_mass],
                                     'surface_growth_mass' : [soot.surface_growth_mass],
                                     'oxidation_mass': [soot.oxidation_mass],
                                     'total_carbon_mass' : [constUV.total_carbon_mass],
                                     'total_hydrogen_mass' : [constUV.total_hydrogen_mass],
                                     'soot_carbon_mass' : [constUV.soot_carbon_mass],
                                        });

sectional_data = None;
if particle_dynamics_model_type == "Sectional":
    n_secs = pdynamics.number_of_sections;
    sectional_header = (
        [f"N_agg_{i}" for i in range(n_secs)] + [f"N_pri_{i}" for i in range(n_secs)] 
        + [f"n_p_{i}" for i in range(n_secs)] + [f"d_p_{i}" for i in range(n_secs)] 
        + [f"d_m_{i}" for i in range(n_secs)] + [f"d_g_{i}" for i in range(n_secs)]
    );
    sectional_data = [[] for _ in sectional_header];
    append_sectional();

In [35]:
constUV.max_step = 5e-4;
constUV.start();
pdynamics.spacing_factor = 1.5 # number of sections = 60 (immutable)

In [38]:
start_time = time.time()
step = 0;
profiles = defaultdict(list)
while constUV.restime < max_res_time:
    step += 1;
    constUV.step();
    profiles["time"].append(constUV.restime)
    profiles["pressure"].append(gas.P)
    profiles["temperature"].append(gas.T)
    profiles["mole_fractions"].append(gas.X)
    log = f"z={constUV.restime:0.5e} s \n"+\
      f"T={constUV.T:0.3f} K\n"+\
      f"soot_mass[ug/g]={(soot.total_mass):0.5e}\n"+\
      40*"-"  ;
    print(log)
    rho = soot_gas.density
    if step%1==0:
        print(log)
        append();
        
end_time = time.time()

z=3.45177e-10 s 
T=1800.000 K
soot_mass[ug/g]=1.77548e-02
----------------------------------------
z=3.45177e-10 s 
T=1800.000 K
soot_mass[ug/g]=1.77548e-02
----------------------------------------
z=6.90354e-10 s 
T=1800.000 K
soot_mass[ug/g]=1.77548e-02
----------------------------------------
z=6.90354e-10 s 
T=1800.000 K
soot_mass[ug/g]=1.77548e-02
----------------------------------------
z=1.36742e-09 s 
T=1800.000 K
soot_mass[ug/g]=1.77548e-02
----------------------------------------
z=1.36742e-09 s 
T=1800.000 K
soot_mass[ug/g]=1.77548e-02
----------------------------------------
z=2.04450e-09 s 
T=1800.000 K
soot_mass[ug/g]=1.77548e-02
----------------------------------------
z=2.04450e-09 s 
T=1800.000 K
soot_mass[ug/g]=1.77548e-02
----------------------------------------
z=2.72157e-09 s 
T=1800.000 K
soot_mass[ug/g]=1.77548e-02
----------------------------------------
z=2.72157e-09 s 
T=1800.000 K
soot_mass[ug/g]=1.77548e-02
----------------------------------------
z=4.34177e

In [17]:
print(f"the simulation time is {end_time-start_time}")

the simulation time is 0.9426274299621582


In [18]:
soot_columns = [];
soot_data = [];
flow_columns = [];
flow_data = [];
species_columns = [];
species_data = [];

soot_columns += ["N_agg[mol/kg]", "N_pri[mol/kg]", "C_tot[mol/kg]", "H_tot[mol/kg]"];
soot_data += [states.N_agg, states.N_pri, states.C_tot, states.H_tot];

soot_columns += ["A_tot[m2/kg]", "N_agg[#/cm3]", "N_pri[#/cm3]", "N_agg[#/g]", "N_pri[#/g]"]
soot_data += [states.A_tot, states.N_agg*Av*states.density/1e6, states.N_pri*Av*states.density/1e6,
             states.N_agg*Av/1e3, states.N_pri*Av/1e3];

soot_columns += ["d_p[nm]", "d_m[nm]", "d_g[nm]", "d_v[nm]", "n_p"]
soot_data += [states.d_p*1e9, states.d_m*1e9, states.d_g*1e9, states.d_v*1e9, states.n_p];

soot_columns += ["soot_mass[kg/kg]", "volume_fraction", "SSA",
                 "total_carbon_mass[kg/kg]", "carbon_yield", "total_carbon_mass[kg/m3]",
                "total_hydrogen_mass[kg/kg]", "total_hydrogen_mass[kg/m3]"];
soot_data += [states.total_mass, states.volume_fraction, states.SSA,
              states.total_carbon_mass, 
              (states.soot_carbon_mass*states.density)/(states.total_carbon_mass[0]*states.density[0]),
             states.total_carbon_mass*states.density,
             states.total_hydrogen_mass, 
             states.total_hydrogen_mass*states.density];

soot_columns += ['inception_mass[mol/kg-s]', "coagulation_mass[mol/kg-s]",
                 "PAH_adsorption_mass[mol/kg-s]", "surface_growth_mass[mol/kg-s]"];
soot_data += [states.inception_mass, states.coagulation_mass,
              states.PAH_adsorption_mass, states.surface_growth_mass];

if sectional_data:
    soot_columns += sectional_header;
    soot_data += sectional_data;

flow_columns += ["t", "T", "density", "mean_MW"];
flow_data += [states.restime ,states.T, states.density, states.mean_molecular_weight/1000];

species_columns = [f"{sp}" for sp in gas.species_names];
species_data += [states.X[:,i] for i in range(len(gas.species_names))]

In [19]:
columns = flow_columns + soot_columns + species_columns;
data = (np.array(flow_data + soot_data + species_data)).transpose();

In [20]:
output_dir = f"results/{PAH_growth_model_type}";
if not os.path.exists(output_dir):
    os.makedirs(output_dir);
prop_df = pd.DataFrame(data=data, columns = columns)
name_suffix = "h_carbon" if constUV.consider_soot_energy else ""
prop_df.to_csv(f"{output_dir}/results{name_suffix}.csv");

### Energy Balance for unit of reactor volume

In [21]:
U1 = states.density[0] * states.int_energy_mass[0];
U2 = states.density[-1] * states.int_energy_mass[-1];
print(f"The difference in the internal energy of gas is {(U2-U1):0.5e} J/kg")

The difference in the internal energy of gas is -2.36314e-03 J/kg


In [22]:
Up = (soot.total_mass*states.density[-1]) * (SootThermo.u_mass_soot(states.T[-1]));
print(f"The internal energy of generated particles is {(Up):0.5e} J/kg")

The internal energy of generated particles is 1.96597e-18 J/kg


In [23]:
print(f"The energy residual is {(U2-U1+Up):0.5e} J/kg")

The energy residual is -2.36314e-03 J/kg
